# ➡️ Forward Propagation - Comment un Réseau Fait des Prédictions

Dans ce notebook, nous allons comprendre en profondeur comment un réseau de neurones fait des **prédictions**.

## 🎯 Objectifs

1. **Comprendre la forward propagation** pas à pas
2. **Implémenter** chaque étape en Python
3. **Visualiser** ce qui se passe dans le réseau
4. **Tester** avec de vraies données MNIST

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent))

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Fixer la seed pour la reproductibilité
np.random.seed(42)

print("✅ Imports réussis !")

## 1️⃣ Rappel : Architecture du Réseau

Nous allons construire un réseau simple pour la classification MNIST :

```
Input Layer       Hidden Layer      Output Layer
   (784)    →       (128)      →       (10)
    
    x        →   W1, b1, ReLU  →    W2, b2, Softmax  →  probabilities
```

### 🔑 Composants clés :

- **Input** : 784 features (28×28 pixels)
- **Hidden Layer** : 128 neurones avec activation ReLU
- **Output Layer** : 10 neurones (un par chiffre) avec activation Softmax


## 2️⃣ Les Opérations Mathématiques

La forward propagation consiste en une série d'opérations matricielles simples !

### Pour chaque couche :

#### Étape 1: Multiplication matricielle + biais
$$Z = X \cdot W + b$$

#### Étape 2: Fonction d'activation
$$A = f(Z)$$

### Pour notre réseau complet :

```
Layer 1 (Hidden):
   Z1 = X · W1 + b1
   A1 = ReLU(Z1)

Layer 2 (Output):
   Z2 = A1 · W2 + b2
   A2 = Softmax(Z2)
   
Output: A2 = probabilités pour chaque classe
```

## 3️⃣ Implémentons les Fonctions d'Activation

Commençons par implémenter les fonctions d'activation que nous allons utiliser.

In [ ]:
def relu(Z):
    """
    ReLU (Rectified Linear Unit)
    
    Formule: f(x) = max(0, x)
    
    Pourquoi ReLU ?
    - Simple et rapide à calculer
    - Évite le problème du gradient qui disparaît
    - Très efficace en pratique
    """
    return np.maximum(0, Z)

def softmax(Z):
    """
    Softmax - Transforme des scores en probabilités
    
    Formule: softmax(x_i) = exp(x_i) / sum(exp(x_j))
    
    Propriétés:
    - Sortie entre 0 et 1 pour chaque classe
    - La somme de toutes les sorties = 1
    - Idéal pour la classification multi-classes
    
    Astuce: On soustrait le max pour la stabilité numérique
    """
    # Soustraire le max pour éviter l'overflow (astuce numérique)
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

# Test des fonctions
print("🧪 Test de ReLU:")
test_relu = np.array([-2, -1, 0, 1, 2])
print(f"   Input:  {test_relu}")
print(f"   Output: {relu(test_relu)}")
print("   ➡️ Les valeurs négatives deviennent 0, les positives restent inchangées\n")

print("🧪 Test de Softmax:")
test_softmax = np.array([[1, 2, 3, 4, 5]])
result_softmax = softmax(test_softmax)
print(f"   Input:  {test_softmax[0]}")
print(f"   Output: {result_softmax[0]}")
print(f"   Somme:  {result_softmax.sum():.4f} (doit être 1.0)")
print("   ➡️ Chaque valeur est une probabilité, somme = 1.0")

### 📊 Visualisons ces Fonctions

In [ ]:
def visualize_activations():
    """
    Visualise ReLU et Softmax
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # ReLU
    x = np.linspace(-3, 3, 100)
    y_relu = relu(x)
    
    ax1.plot(x, y_relu, linewidth=3, color='#e74c3c', label='ReLU(x)')
    ax1.plot(x, x, '--', linewidth=2, color='gray', alpha=0.5, label='y = x (référence)')
    ax1.axhline(y=0, color='black', linewidth=0.8)
    ax1.axvline(x=0, color='black', linewidth=0.8)
    ax1.fill_between(x, 0, y_relu, alpha=0.3, color='#e74c3c')
    ax1.set_xlabel('Input (Z)', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Output', fontsize=13, fontweight='bold')
    ax1.set_title('⚡ Fonction ReLU', fontsize=16, fontweight='bold')
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Annotations
    ax1.annotate('Zone morte\n(gradient = 0)', xy=(-2, 0), xytext=(-2.5, 0.8),
                fontsize=11, ha='center',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', lw=2))
    ax1.annotate('Zone active\n(gradient = 1)', xy=(2, 2), xytext=(1.5, 2.8),
                fontsize=11, ha='center',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7),
                arrowprops=dict(arrowstyle='->', lw=2))
    
    # Softmax - exemple avec 5 classes
    scores = np.array([[1, 2, 3, 4, 5],
                       [5, 4, 3, 2, 1],
                       [3, 3, 3, 3, 3],
                       [1, 5, 1, 1, 1]])
    probabilities = softmax(scores)
    
    x_pos = np.arange(4)
    width = 0.15
    colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c', '#9b59b6']
    
    for class_idx in range(5):
        ax2.bar(x_pos + class_idx * width, probabilities[:, class_idx], 
               width, label=f'Classe {class_idx}', color=colors[class_idx],
               edgecolor='black', linewidth=1)
    
    ax2.set_xlabel('Exemple', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Probabilité', fontsize=13, fontweight='bold')
    ax2.set_title('🎯 Fonction Softmax (5 classes)', fontsize=16, fontweight='bold')
    ax2.set_xticks(x_pos + width * 2)
    ax2.set_xticklabels(['Ex 1', 'Ex 2', 'Ex 3', 'Ex 4'])
    ax2.legend(fontsize=10, loc='upper right')
    ax2.grid(axis='y', alpha=0.3)
    ax2.set_ylim(0, 1)
    
    # Annotations
    ax2.axhline(y=1.0, color='red', linestyle='--', linewidth=2, alpha=0.5)
    ax2.text(3.5, 1.02, 'Somme = 1.0 pour chaque exemple', 
            fontsize=11, ha='right', style='italic', color='red')
    
    plt.tight_layout()
    plt.show()

visualize_activations()

## 4️⃣ Initialisation des Poids

Avant de faire des prédictions, il faut initialiser les poids du réseau.

### 🎲 Stratégies d'initialisation :

- **Aléatoire simple** : Mauvaise idée (gradients peuvent exploser/disparaître)
- **Xavier/Glorot** : Bon pour sigmoid/tanh
- **He initialization** : ⭐ Idéal pour ReLU (ce que nous utilisons)


In [ ]:
def initialize_parameters(layer_dims):
    """
    Initialise les poids et biais du réseau
    
    Args:
        layer_dims: liste des dimensions de chaque couche
                   ex: [784, 128, 10] pour notre réseau
    
    Returns:
        parameters: dictionnaire contenant W1, b1, W2, b2, etc.
    """
    np.random.seed(42)
    parameters = {}
    L = len(layer_dims)  # nombre de couches
    
    for l in range(1, L):
        # He initialization: multiplier par sqrt(2/n)
        # Optimisé pour ReLU
        parameters[f'W{l}'] = np.random.randn(layer_dims[l-1], layer_dims[l]) * np.sqrt(2 / layer_dims[l-1])
        
        # Biais initialisés à zéro
        parameters[f'b{l}'] = np.zeros((1, layer_dims[l]))
    
    return parameters

# Créer notre architecture
layer_dimensions = [784, 128, 10]
params = initialize_parameters(layer_dimensions)

print("🏗️ Architecture du réseau créée !\n")
print("="*60)
print("📊 PARAMÈTRES DU RÉSEAU")
print("="*60)

for key, value in params.items():
    print(f"\n{key}: shape = {value.shape}")
    if 'W' in key:
        print(f"   - Nombre de paramètres: {value.size:,}")
        print(f"   - Moyenne: {value.mean():.6f}")
        print(f"   - Std: {value.std():.6f}")

total_params = sum(p.size for p in params.values())
print(f"\n{'='*60}")
print(f"✨ Nombre total de paramètres: {total_params:,}")
print(f"   (C'est ce que le réseau va apprendre !)")

### 📊 Visualisons la Distribution des Poids Initiaux

In [ ]:
def visualize_weight_distribution(parameters):
    """
    Visualise la distribution des poids après initialisation
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('📊 Distribution des Poids Initiaux (He Initialization)', 
                fontsize=18, fontweight='bold')
    
    # W1
    axes[0].hist(parameters['W1'].flatten(), bins=50, color='#3498db', 
                edgecolor='black', alpha=0.7)
    axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
    axes[0].set_xlabel('Valeur du poids', fontsize=13, fontweight='bold')
    axes[0].set_ylabel('Fréquence', fontsize=13, fontweight='bold')
    axes[0].set_title(f'W1: {parameters["W1"].shape}', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Stats
    mean_w1 = parameters['W1'].mean()
    std_w1 = parameters['W1'].std()
    axes[0].text(0.95, 0.95, f'μ = {mean_w1:.4f}\nσ = {std_w1:.4f}',
                transform=axes[0].transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # W2
    axes[1].hist(parameters['W2'].flatten(), bins=50, color='#2ecc71',
                edgecolor='black', alpha=0.7)
    axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Valeur du poids', fontsize=13, fontweight='bold')
    axes[1].set_ylabel('Fréquence', fontsize=13, fontweight='bold')
    axes[1].set_title(f'W2: {parameters["W2"].shape}', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Stats
    mean_w2 = parameters['W2'].mean()
    std_w2 = parameters['W2'].std()
    axes[1].text(0.95, 0.95, f'μ = {mean_w2:.4f}\nσ = {std_w2:.4f}',
                transform=axes[1].transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Pourquoi cette distribution ?")
    print("   • Centrée sur 0 (μ ≈ 0)")
    print("   • Variance calculée pour éviter explosion/disparition du gradient")
    print("   • He initialization: optimale pour ReLU")

visualize_weight_distribution(params)

## 5️⃣ Forward Propagation - Implémentation Complète

Maintenant, implémentons la forward propagation complète !

In [ ]:
def forward_propagation(X, parameters, verbose=False):
    """
    Effectue la forward propagation
    
    Args:
        X: données d'entrée (n_samples, 784)
        parameters: poids et biais du réseau
        verbose: afficher les détails étape par étape
    
    Returns:
        A2: prédictions finales (probabilités)
        cache: valeurs intermédiaires (utiles pour la backprop)
    """
    # Récupérer les paramètres
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    if verbose:
        print("\n" + "="*70)
        print("🚀 FORWARD PROPAGATION - ÉTAPE PAR ÉTAPE")
        print("="*70)
        print(f"\n📥 Input: X.shape = {X.shape}")
    
    # ===== COUCHE 1 (Hidden Layer) =====
    # Étape 1a: Multiplication matricielle + biais
    Z1 = np.dot(X, W1) + b1
    if verbose:
        print(f"\n🔸 Layer 1 - Linear Transformation:")
        print(f"   Z1 = X · W1 + b1")
        print(f"   Z1.shape = {Z1.shape}")
        print(f"   Z1 range: [{Z1.min():.3f}, {Z1.max():.3f}]")
    
    # Étape 1b: Activation ReLU
    A1 = relu(Z1)
    if verbose:
        print(f"\n🔸 Layer 1 - Activation (ReLU):")
        print(f"   A1 = ReLU(Z1)")
        print(f"   A1.shape = {A1.shape}")
        print(f"   A1 range: [{A1.min():.3f}, {A1.max():.3f}]")
        print(f"   Neurons activated: {np.sum(A1 > 0)} / {A1.size} ({np.sum(A1 > 0)/A1.size*100:.1f}%)")
    
    # ===== COUCHE 2 (Output Layer) =====
    # Étape 2a: Multiplication matricielle + biais
    Z2 = np.dot(A1, W2) + b2
    if verbose:
        print(f"\n🔸 Layer 2 - Linear Transformation:")
        print(f"   Z2 = A1 · W2 + b2")
        print(f"   Z2.shape = {Z2.shape}")
        print(f"   Z2 range: [{Z2.min():.3f}, {Z2.max():.3f}]")
    
    # Étape 2b: Activation Softmax
    A2 = softmax(Z2)
    if verbose:
        print(f"\n🔸 Layer 2 - Activation (Softmax):")
        print(f"   A2 = Softmax(Z2)")
        print(f"   A2.shape = {A2.shape}")
        print(f"   A2 range: [{A2.min():.3f}, {A2.max():.3f}]")
        print(f"   Sum of probabilities: {A2[0].sum():.6f} (should be 1.0)")
        print("\n" + "="*70)
        print("✅ FORWARD PROPAGATION COMPLETE")
        print("="*70)
    
    # Sauvegarder les valeurs pour la backpropagation
    cache = {
        'Z1': Z1, 'A1': A1,
        'Z2': Z2, 'A2': A2
    }
    
    return A2, cache

print("✅ Fonction forward_propagation créée !")

## 6️⃣ Test avec des Données Réelles

Testons notre forward propagation avec de vraies images MNIST !

In [ ]:
# Charger MNIST
from src.utils import load_mnist_data

X_train, y_train, X_test, y_test = load_mnist_data()

# Prendre un seul exemple
single_example = X_train[0:1]  # Shape: (1, 784)
true_label = y_train[0]

print(f"\n🖼️ Test avec une image du chiffre: {true_label}")

# Forward propagation avec mode verbose
predictions, cache = forward_propagation(single_example, params, verbose=True)

### 📊 Visualisons la Prédiction

In [ ]:
def visualize_prediction(image, predictions, true_label):
    """
    Visualise l'image et les prédictions du réseau
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Image originale
    ax1.imshow(image.reshape(28, 28), cmap='gray_r')
    ax1.set_title(f'🖼️ Image du chiffre: {true_label}', 
                 fontsize=16, fontweight='bold')
    ax1.axis('off')
    
    # Prédictions
    classes = range(10)
    probs = predictions[0]
    predicted_class = np.argmax(probs)
    
    colors = ['green' if i == predicted_class else 'lightblue' for i in classes]
    colors[true_label] = 'orange' if true_label != predicted_class else 'green'
    
    bars = ax2.bar(classes, probs, color=colors, edgecolor='black', linewidth=2, alpha=0.8)
    ax2.set_xlabel('Classe (Chiffre)', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Probabilité', fontsize=13, fontweight='bold')
    ax2.set_title('🎯 Prédictions du Réseau (non entraîné)', 
                 fontsize=16, fontweight='bold')
    ax2.set_xticks(classes)
    ax2.set_ylim(0, 1)
    ax2.grid(axis='y', alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for i, (bar, prob) in enumerate(zip(bars, probs)):
        height = bar.get_height()
        if prob > 0.01:  # Afficher seulement si > 1%
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{prob:.1%}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Légende
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='green', label='Prédiction (et vrai label)' if predicted_class == true_label else 'Prédiction'),
        Patch(facecolor='orange', label='Vrai label'),
        Patch(facecolor='lightblue', label='Autres classes')
    ]
    if predicted_class == true_label:
        legend_elements = legend_elements[:1] + legend_elements[2:]
    ax2.legend(handles=legend_elements, fontsize=11, loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # Résultat
    print("\n" + "="*70)
    print("📊 RÉSULTAT DE LA PRÉDICTION")
    print("="*70)
    print(f"\n🎯 Vrai label: {true_label}")
    print(f"🤖 Prédiction: {predicted_class}")
    print(f"📈 Confiance: {probs[predicted_class]:.2%}")
    
    if predicted_class == true_label:
        print(f"\n✅ CORRECT ! (mais c'est par hasard, le réseau n'est pas encore entraîné)")
    else:
        print(f"\n❌ INCORRECT (normal, le réseau n'est pas encore entraîné)")
    
    print(f"\n📊 Top 3 prédictions:")
    top3_indices = np.argsort(probs)[::-1][:3]
    for i, idx in enumerate(top3_indices, 1):
        print(f"   {i}. Chiffre {idx}: {probs[idx]:.2%}")

visualize_prediction(single_example, predictions, true_label)

## 7️⃣ Testons avec Plusieurs Exemples

In [ ]:
def test_multiple_predictions(X, y, parameters, n_samples=10):
    """
    Teste le réseau sur plusieurs exemples
    """
    # Sélectionner des exemples aléatoires
    indices = np.random.choice(len(X), n_samples, replace=False)
    
    # Forward propagation
    predictions, _ = forward_propagation(X[indices], parameters)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = y[indices]
    
    # Visualiser
    fig, axes = plt.subplots(2, 5, figsize=(16, 7))
    fig.suptitle('🎲 Prédictions sur 10 Exemples Aléatoires (Réseau Non Entraîné)', 
                fontsize=18, fontweight='bold')
    
    for idx, ax in enumerate(axes.flat):
        image = X[indices[idx]].reshape(28, 28)
        true_label = true_labels[idx]
        pred_label = predicted_labels[idx]
        confidence = predictions[idx][pred_label]
        
        ax.imshow(image, cmap='gray_r')
        
        # Couleur selon correct/incorrect
        color = 'green' if pred_label == true_label else 'red'
        title = f'Vrai: {true_label} | Pred: {pred_label}\nConf: {confidence:.1%}'
        ax.set_title(title, fontsize=11, fontweight='bold', color=color)
        ax.axis('off')
        
        # Bordure
        for spine in ax.spines.values():
            spine.set_edgecolor(color)
            spine.set_linewidth(3)
            spine.set_visible(True)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques
    accuracy = np.mean(predicted_labels == true_labels)
    print("\n" + "="*70)
    print("📊 STATISTIQUES")
    print("="*70)
    print(f"\n🎯 Accuracy: {accuracy:.1%} ({np.sum(predicted_labels == true_labels)}/{n_samples})")
    print(f"\n💡 Note: Le réseau n'est PAS encore entraîné !")
    print(f"   - Performance aléatoire attendue: ~10% (1/10 classes)")
    print(f"   - Performance actuelle: {accuracy:.1%}")
    if accuracy > 0.15:
        print(f"   ➡️ Un peu de chance ! 🍀")
    else:
        print(f"   ➡️ Normal pour un réseau non entraîné")

test_multiple_predictions(X_test, y_test, params)

## 8️⃣ Visualisons l'Activité des Neurones

Regardons ce qui se passe dans la couche cachée !

In [ ]:
def visualize_hidden_layer_activity(X, y, parameters, sample_idx=0):
    """
    Visualise l'activité de la couche cachée
    """
    # Forward propagation
    single_input = X[sample_idx:sample_idx+1]
    _, cache = forward_propagation(single_input, parameters)
    
    A1 = cache['A1'][0]  # Activations de la couche cachée (128 neurones)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'🔬 Analyse de la Couche Cachée - Chiffre: {y[sample_idx]}', 
                fontsize=18, fontweight='bold')
    
    # 1. Image originale
    axes[0, 0].imshow(X[sample_idx].reshape(28, 28), cmap='gray_r')
    axes[0, 0].set_title('🖼️ Image d\'entrée', fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')
    
    # 2. Activations des neurones
    axes[0, 1].bar(range(128), A1, color='steelblue', edgecolor='black', linewidth=0.5, alpha=0.7)
    axes[0, 1].set_xlabel('Neurone', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Activation', fontsize=12, fontweight='bold')
    axes[0, 1].set_title(f'⚡ Activations des 128 Neurones', fontsize=14, fontweight='bold')
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # 3. Distribution des activations
    axes[1, 0].hist(A1, bins=30, color='coral', edgecolor='black', alpha=0.7)
    axes[1, 0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zéro (ReLU coupure)')
    axes[1, 0].set_xlabel('Valeur d\'activation', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Fréquence', fontsize=12, fontweight='bold')
    axes[1, 0].set_title('📊 Distribution des Activations', fontsize=14, fontweight='bold')
    axes[1, 0].legend(fontsize=11)
    axes[1, 0].grid(axis='y', alpha=0.3)
    
    # 4. Statistiques
    axes[1, 1].axis('off')
    
    active_neurons = np.sum(A1 > 0)
    inactive_neurons = np.sum(A1 == 0)
    stats_text = f"""
    📈 STATISTIQUES DE LA COUCHE CACHÉE
    {'='*50}
    
    🔹 Neurones actifs: {active_neurons} / 128 ({active_neurons/128*100:.1f}%)
    🔹 Neurones inactifs: {inactive_neurons} / 128 ({inactive_neurons/128*100:.1f}%)
    
    🔹 Valeurs:
       • Minimum: {A1.min():.4f}
       • Maximum: {A1.max():.4f}
       • Moyenne: {A1.mean():.4f}
       • Médiane: {np.median(A1):.4f}
       • Écart-type: {A1.std():.4f}
    
    🔹 Top 5 neurones les plus actifs:
    """
    
    top5_indices = np.argsort(A1)[::-1][:5]
    for i, idx in enumerate(top5_indices, 1):
        stats_text += f"\n       {i}. Neurone {idx}: {A1[idx]:.4f}"
    
    stats_text += "\n\n    💡 Note: Ces activations sont aléatoires car"
    stats_text += "\n       le réseau n'est pas encore entraîné !"
    
    axes[1, 1].text(0.1, 0.5, stats_text, transform=axes[1, 1].transAxes,
                   fontsize=11, verticalalignment='center', family='monospace',
                   bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Visualiser pour un exemple aléatoire
random_idx = np.random.randint(0, 100)
visualize_hidden_layer_activity(X_test, y_test, params, random_idx)

## 🎯 Récapitulatif

Dans ce notebook, nous avons appris la **Forward Propagation** en profondeur !

### ✅ Ce que nous avons couvert

1. **Les opérations mathématiques** :
   - Multiplication matricielle: `Z = X · W + b`
   - Fonctions d'activation: ReLU et Softmax

2. **L'initialisation des poids** :
   - He initialization pour ReLU
   - Importance de la bonne initialisation

3. **L'implémentation complète** :
   - Forward propagation étape par étape
   - Test avec de vraies données MNIST

4. **La visualisation** :
   - Prédictions du réseau
   - Activité des neurones cachés
   - Distribution des activations

### 🧠 Points clés

- 📊 **Forward propagation** = comment le réseau fait des prédictions
- 🎲 **Poids aléatoires** = prédictions aléatoires (~10% accuracy)
- 📈 **ReLU** met à zéro ~50% des neurones (c'est normal !)
- 🎯 **Softmax** transforme les scores en probabilités (somme = 1)

### 💡 Prochaine Étape

Maintenant nous savons comment le réseau fait des prédictions.
Mais comment **apprend-il** ?

**➡️ Prochain notebook: `03_backpropagation.ipynb`**

Nous allons découvrir la **backpropagation** - l'algorithme qui permet au réseau d'apprendre ! 🎓

---

**Super travail ! Continue comme ça ! 🚀**